In [76]:
import argparse
import time
import torch
print(torch.__version__)
from Models import get_model
from Process import *
import torch.nn.functional as F
from Optim import CosineWithRestarts
from Batch import create_masks
import dill as pickle
import time
import math
import numpy as np
import multiprocessing as mp
import random
import string
import sys
import os
import whoosh, glob, time, pickle
import whoosh.fields as wf
from whoosh.qparser import QueryParser
from whoosh import index
import threading
from whoosh import filedb
from whoosh.filedb.filestore import FileStorage
import numpy as np
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import multiprocessing as mp
import time
from InMemorySearch import *
from client import *
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from logger import Logger
logger = Logger('./logs')

1.0.0


In [77]:
#At first checking our super clients
# query_list = ["european crime records", "crime records", "european crime", "european records", "crime crimes violent sheriff enforcement re criminals stresak bill strikes", "LA times corpus", "crime records"]    
# super_client = SuperClient()
# print(super_client.hosts)
# final_result = super_client.query_expansion_distributed(query_list)
# print(len(final_result))

In [78]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [79]:
parser = argparse.ArgumentParser()
#If we are working on small dataset
small = 0
#If we want to activate relevance based training
relevance_training = 0
    
if small==1:
    parser.add_argument('-src_data', type=str, default='data/italian_small.txt')
    parser.add_argument('-trg_data', type=str, default='data/english_small.txt')
    parser.add_argument('-trg_data_retrieval', type=str, default='data/english_retrieval.txt')

else:
    parser.add_argument('-src_data', type=str, default='data/italian.txt')
    parser.add_argument('-trg_data', type=str, default='data/english.txt')  
    parser.add_argument('-trg_data_retrieval', type=str, default='data/LATIMESTEXT2.txt')
    parser.add_argument('-rm_data', type=str, default='data/english_rm.txt') ##

parser.add_argument('-src_lang', type=str, default='it')
parser.add_argument('-trg_lang', type=str, default='en')
parser.add_argument('-no_cuda', action='store_true')
parser.add_argument('-SGDR', action='store_true')
parser.add_argument('-epochs', type=int, default=10)
parser.add_argument('-d_model', type=int, default=200)
parser.add_argument('-n_layers', type=int, default=6)
parser.add_argument('-heads', type=int, default=8)
parser.add_argument('-dropout', type=int, default=0.1)
parser.add_argument('-batchsize', type=int, default=1000)
parser.add_argument('-printevery', type=int, default=5)
parser.add_argument('-load_vocab', type=str, default='clir_it_en')

if relevance_training == 1:
    my_file = Path("weights/model_weights")
    if my_file.is_file():
        parser.add_argument('-load_weights', type=str, default='weights')
    else:
        parser.add_argument('-load_weights', type=str, default=None)
    parser.add_argument('-lr', type=int, default=0.01)
else: 
    my_file = Path("weights/model_weights")
    if my_file.is_file():
        parser.add_argument('-load_weights', type=str, default='weights')
    else:
        parser.add_argument('-load_weights', type=str, default=None)         
    parser.add_argument('-lr', type=int, default=0.0001)
    
parser.add_argument('-create_valset', action='store_true')
parser.add_argument('-max_strlen', type=int, default=80)
parser.add_argument('-floyd', action='store_true')
parser.add_argument('-checkpoint', type=int, default=5)

opt = parser.parse_args(args=[])


In [80]:
def tokenizer(text):  # create a tokenizer function
    return text.split()
    
def create_fields(opt):    
    print("loading tokenizers...") 
    TRG = data.Field(lower=True, tokenize=tokenizer, init_token='<sos>', eos_token='<eos>')
    TRG_REL = data.Field(lower=True, tokenize=tokenizer, init_token='<sos>', eos_token='<eos>')    
    #RM = data.Field(lower=True, tokenize=tokenizer, init_token='<sos>', eos_token='<eos>') ##     
    SRC = data.Field(lower=True, tokenize=tokenizer)   
    SRC = pickle.load(open(f'{opt.load_vocab}/SRC.pkl', 'rb'))
    TRG = pickle.load(open(f'{opt.load_vocab}/TRG.pkl', 'rb'))
    TRG_REL = pickle.load(open(f'{opt.load_vocab}/TRG.pkl', 'rb'))
    return(SRC, TRG, TRG_REL)

#this function will consider both europarl and CLEF
def create_dataset(opt, SRC, TRG, TRG_REL):
    print("creating dataset and iterator... ")
    translation_data = [line for line in opt.trg_data] ###
    relevance_data = [line for line in open(opt.rm_data)] ###    
    
    raw_data = {'src' : [line.strip() for line in opt.src_data], 'trg': translation_data, 'trg_rel': relevance_data} ###  
    raw_data_retrieval = {'trg': [line for line in opt.trg_data_retrieval]}
    
    df = pd.DataFrame(raw_data, columns=["src", "trg", "trg_rel"])
    
    vectorizer = TfidfVectorizer(use_idf=True)
    vectorizer.fit_transform(raw_data['trg'] + raw_data_retrieval['trg'])
    
    tokens = vectorizer.get_feature_names()
    idf_values = vectorizer.idf_
    opt.idf_dict = {}
     
    for i in range(len(tokens)):
        opt.idf_dict.setdefault(tokens[i],idf_values[i]) 
    
    mask = (df['src'].str.count(' ') < opt.max_strlen) & (df['trg'].str.count(' ') < opt.max_strlen)
    df = df.loc[mask]

    df.to_csv("translate_transformer_temp.csv", index=False)

    data_fields = [('src', SRC), ('trg', TRG) , ('trg_rel', TRG_REL)]
    
    train = data.TabularDataset('./translate_transformer_temp.csv', format='csv', fields=data_fields)

    train_iter = MyIterator(train, batch_size=opt.batchsize, device=opt.device,
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                        batch_size_fn=batch_size_fn, train=True, shuffle=True)
      
    os.remove('translate_transformer_temp.csv')   
    
    if opt.load_vocab is None:
        print("creating dataset for retrieval corpus... ")
        raw_data = {'trg': [line for line in opt.trg_data_retrieval]}
        df = pd.DataFrame(raw_data, columns=["trg"])
        mask = (df['trg'].str.count(' ') > 1)
        df = df.loc[mask]
        df.to_csv("translate_transformer_retrieval_temp.csv", index=False)
        data_fields = [('trg', TRG)]
        train_retrieval = data.TabularDataset('./translate_transformer_retrieval_temp.csv', format='csv', fields=data_fields)
        os.remove('translate_transformer_retrieval_temp.csv')    

        print("building vocabulary for both europarl and retrieval corpus")

        SRC.build_vocab(train)
        TRG.build_vocab(train, train_retrieval)

    opt.src_pad = SRC.vocab.stoi['<pad>']
    opt.trg_pad = TRG.vocab.stoi['<pad>']
    opt.trg_pad = TRG.vocab.stoi['<pad>']
    
    opt.train_len = get_len(train_iter)
    return train_iter



In [81]:
opt.device = 0 if opt.no_cuda is False else -1
if opt.device == 0:
    assert torch.cuda.is_available()
print(opt.device)
read_data(opt)

SRC, TRG, TRG_REL = create_fields(opt)
opt.train = create_dataset(opt, SRC, TRG, TRG_REL)
dst = ''
#TRG = create_retrieval_vocabulary(opt, TRG)

if opt.checkpoint > 0:
    print("model weights will be saved every %d minutes and at end of epoch to directory weights/"%(opt.checkpoint))
    
if opt.load_vocab is None:
    pickle.dump(SRC, open(f'{dst}/SRC.pkl', 'wb'))
    pickle.dump(TRG, open(f'{dst}/TRG.pkl', 'wb'))

0
loading tokenizers...
creating dataset and iterator... 
1894217
1894217


The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


model weights will be saved every 5 minutes and at end of epoch to directory weights/


In [82]:
model = get_model(opt, len(SRC.vocab), len(TRG.vocab))
opt.optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr, betas=(0.9, 0.98), eps=1e-9)

if opt.SGDR == True:
    opt.sched = CosineWithRestarts(opt.optimizer, T_max=opt.train_len)
    
import subprocess 
super_client = SuperClient()
print(TRG.vocab.itos[0])
print(len(TRG.vocab))
opt.idf_dict["<sos>"] = 1
opt.idf_dict["<eos>"] = 1
class_weights = []
for i in range(len(TRG.vocab)):
    if TRG.vocab.itos[i] in opt.idf_dict:
        class_weights.append(opt.idf_dict[TRG.vocab.itos[i]])
    else:
        class_weights.append(0.0001)
class_weights = torch.FloatTensor(class_weights)

loading pretrained weights...
<unk>
240229


In [84]:
opt.checkpoint = 1
def train_model(model, opt, SRC, TRG):
    torch.cuda.empty_cache()
    best_mAP = 0.15
    inmem = WhooshInMemorySearch()
    print("training model...")
    opt.idf_dict["<sos>"] = 1
    opt.idf_dict["<eos>"] = 1
    class_weights = []
    for i in range(len(TRG.vocab)):
        if TRG.vocab.itos[i] in opt.idf_dict:
            class_weights.append(opt.idf_dict[TRG.vocab.itos[i]])            
        else:
            class_weights.append(0.0001)
    
    class_weights = torch.FloatTensor(class_weights)
    model.train()
    class_weights = class_weights.cuda()
    start = time.time()
    if opt.checkpoint > 0:
        cptime = time.time()
    
    global_step = 0
    for epoch in range(opt.epochs):
        #print("beginning epoch:")
        total_loss = 0
        
        #print("   %dm: epoch %d [%s]  %d%%  loss = %s" %\
        #((time.time() - start)//60, epoch + 1, "".join(' '*20), 0, '...'), end='\r')

        if opt.checkpoint > 0:
            torch.save(model.state_dict(), 'weights/model_weights')
                    
        for i, batch in enumerate(opt.train):
            torch.cuda.empty_cache()
            start = time.clock()
            src = batch.src.transpose(0,1) # src_size = (187, 4)
            trg = batch.trg.transpose(0,1) # trg_size = (187, 8)             
            trg_rel = batch.trg_rel.transpose(0,1)
#           trg_idf = [[opt.idf_dict[TRG.vocab.itos[ind]] for ind in ex] for ex in trg]
#           trg_idf = np.array(trg_idf)
#           trg_idf = torch.FloatTensor(trg_idf)
            #print (trg_idf.size())
            
            if relevance_training == 1:
                trg_strings = [' '.join([TRG.vocab.itos[ind] for ind in ex]) for ex in trg]               
                trg_strings_rm = super_client.query_expansion_distributed(trg_strings)                
                trg_strings_rm_id = torch.LongTensor([[TRG.vocab.stoi[token] for token in sentence] for sentence in trg_strings_rm]).cuda()            
                embed_trg = model.decoder.embed(trg_strings_rm_id)
                #old loss
                #embed_trg = embed_trg.view(embed_trg.size(0), -1)
                #new loss 
                embed_trg = embed_trg.sum(1)
            
            if relevance_training == 2:
                trg_strings_rm_id = torch.LongTensor([token_id for token_id in token_ids] for token_ids in trg_rel]).cuda()            
                embed_trg = model.decoder.embed(trg_strings_rm_id)
                #old loss
                #embed_trg = embed_trg.view(embed_trg.size(0), -1)
                #new loss 
                embed_trg = embed_trg.sum(1)
            
            
            trg_input = trg[:, :-1]
            src_mask, trg_mask = create_masks(src, trg_input, opt)            
            src_mask = src_mask.cuda()            
            trg_mask = trg_mask.cuda()
            src = src.cuda() 
            trg_input = trg_input.cuda()                                  
            preds = model(src, trg_input, src_mask, trg_mask)
            
            #the goal is to find the embedding of the predictions 
            if relevance_training == 1: 
                out = F.softmax(preds, dim=-1)
                probs, ix = out[:, :].data.topk(1)
                preds_token_ids = ix.view(ix.size(0), -1)
                #new loss function 
                embed_pred = model.decoder.embed(preds_token_ids)
                embed_pred = embed_pred.sum(1)
                #print("embed preds size " + str(embed_pred_sum.size()))            

                #old loss function
#                 pred_strings = [' '.join([TRG.vocab.itos[ind] for ind in ex]) for ex in preds_token_ids]
#                 pred_strings_rm = super_client.query_expansion_distributed(pred_strings)                                
#                 pred_strings_rm_id = torch.LongTensor([[TRG.vocab.stoi[token] for token in sentence] for sentence in pred_strings_rm]).cuda()            
#                 embed_pred = model.decoder.embed(pred_strings_rm_id)
#                 embed_pred = embed_pred.view(embed_pred.size(0), -1)            
                     
            #ys_w = trg_idf[:, 1:].contiguous().view(-1).cuda()
            if relevance_training == 2: 
                out = F.softmax(preds, dim=-1)
                probs, ix = out[:, :].data.topk(1)
                preds_token_ids = ix.view(ix.size(0), -1)
                #new loss function 
                embed_pred = model.decoder.embed(preds_token_ids)
                embed_pred = embed_pred.sum(1)

            ys = trg[:, 1:].contiguous().view(-1).cuda()
            opt.optimizer.zero_grad()
            
            if relevance_training == 1:
                #loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=opt.trg_pad).add(
                #((embed_trg.cuda() - embed_pred.cuda()) **2).mean())
                loss1 = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, weight=ys_w, ignore_index=opt.trg_pad)              
                loss2 = ((embed_trg.cuda() - embed_pred.cuda()) **2).mean()
                
                print("batch loss nmt\t" + str(loss1.item()) + "\tbatch loss relevance\t" + str(loss2.item()))
                loss1.backward(retain_graph=True)
                loss2.backward()      
            if relevance_training == 2:
                loss1 = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, weight=ys_w, ignore_index=opt.trg_pad)              
                loss2 = ((embed_trg.cuda() - embed_pred.cuda()) **2).mean()
                
                print("batch loss nmt\t" + str(loss1.item()) + "\tbatch loss relevance\t" + str(loss2.item()))
                loss1.backward(retain_graph=True)
                loss2.backward()         
            
            else:
                loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, weight=class_weights, ignore_index=opt.trg_pad)              
                #loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=opt.trg_pad)              
                loss.backward()             
            
            opt.optimizer.step()
            
            if opt.SGDR == True: 
                opt.sched.step()
            #total_loss += loss1.item() + loss2.item()
            total_loss+= loss.item()
            
            if (i + 1) % opt.printevery == 0:
                #print("lost after some iterations " + str(total_loss))
                
                p = int(100 * (i + 1) / opt.train_len)
                avg_loss = total_loss/opt.printevery
                
                #print("   %dm: epoch %d [%s%s]  %d%%  loss = %.3f" %\
                #((time.time() - start)//60, epoch + 1, "".join('#'*(p//5)), "".join(' '*(20-(p//5))), p, avg_loss), end='\r')
                #print(avg_loss)
                total_loss = 0
            #We are saving the model every five minutes 
            if opt.checkpoint > 0 and ((time.time()-cptime)//60) // opt.checkpoint >= 1:
                torch.save(model.state_dict(), 'weights/model_weights')                            
                p = subprocess.Popen('CUDA_VISIBLE_DEVICES=2 python translate_validation.py', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
                line = p.stdout.readlines()[0]
                #print(line)
                mAP = float(line.decode("utf-8").strip().split()[1])
                #print("current mAP is\t" + str(mAP))
                #print("best mAP so far is\t" + str(best_mAP))     
                print(str(global_step) + "\t" + str(loss.item()) + "\t" + str(mAP))
                if mAP > best_mAP:
                    #print("best mAP so far is " + str(mAP)) 
                    #print("saving the model at model_weights_best_validation")
                    best_mAP = mAP
                    torch.save(model.state_dict(), 'weights/model_weights_best_validation')
                
                #torch.save(model.state_dict(), 'weights/model_weights')  
                
                cptime = time.time()
                # 1. Log scalar values (scalar summary)
                info = { 'loss': loss.item(), 'map': mAP}
                for tag, value in info.items():
                    logger.scalar_summary(tag, value, global_step+1)
                global_step+=1
        print("%dm: epoch %d [%s%s]  %d%%  loss = %.3f\nepoch %d complete, loss = %.03f" %\
        ((time.time() - start)//60, epoch + 1, "".join('#'*(100//5)), "".join(' '*(20-(100//5))), 100, avg_loss, epoch + 1, avg_loss))
        torch.save(model.state_dict(), 'weights/model_weights_8_' + str(epoch))

In [85]:
#torch.cuda.empty_cache()
model = model.cuda()
train_model(model, opt, SRC, TRG)

training model...
0	3.7884838581085205	0.1225
1	4.234820365905762	0.1164
2	3.9458653926849365	0.1252
3	6.230319499969482	0.1449
4	3.608670234680176	0.1403
5	3.5625908374786377	0.1311
6	3.8527517318725586	0.1347
7	4.155653953552246	0.1222
8	4.225128173828125	0.0879
9	4.1235198974609375	0.0984
10	3.967794418334961	0.1268
11	4.156708240509033	0.1415
12	4.160446643829346	0.1336
13	3.8999078273773193	0.1091
14	3.716236114501953	0.1027
15	4.504022121429443	0.1351
16	3.8989346027374268	0.1444
17	4.1869025230407715	0.1426
18	3.788221597671509	0.1398
19	4.45277738571167	0.094
20	3.871441602706909	0.1389
21	3.6745269298553467	0.137
22	2.9278037548065186	0.1282
23	3.819335699081421	0.1138
24	4.125905513763428	0.1326
25	3.6772305965423584	0.1392
26	3.890035629272461	0.1518
27	3.9780538082122803	0.1411
28	4.294170379638672	0.1271
29	3.6222739219665527	0.1125
30	3.8926122188568115	0.1188
31	4.538330078125	0.1179
32	4.631744384765625	0.1103
33	3.634626865386963	0.1023
34	4.408799648284912	0.1385
35	4

284	4.180558204650879	0.149
285	3.5520238876342773	0.1088
286	3.931853771209717	0.1124
287	3.9226174354553223	0.1156
288	3.5832507610321045	0.1366
289	3.6951212882995605	0.1289
290	3.846214532852173	0.1246
291	3.5974676609039307	0.1269
292	3.8336257934570312	0.1038
293	4.22176456451416	0.136
294	4.41612434387207	0.1252
295	4.144439697265625	0.1213
296	3.6642966270446777	0.1218
297	3.780883312225342	0.1264
298	3.8099570274353027	0.105
299	4.332866668701172	0.1353
300	4.009336948394775	0.1134
301	4.259526252746582	0.1452
302	3.9161629676818848	0.1094
303	4.116866588592529	0.1009
304	4.424443244934082	0.1304
305	4.112651824951172	0.1157
306	4.087150573730469	0.1109
307	4.200462341308594	0.1361
308	3.2811808586120605	0.1141
309	4.295328617095947	0.1267
310	2.2064003944396973	0.1093
311	4.091166019439697	0.1177
312	3.575411796569824	0.1109
313	3.7301833629608154	0.1091
314	4.081258773803711	0.1181
315	3.1733953952789307	0.1012
316	3.768211603164673	0.116
317	3.287606716156006	0.1077
318	4.1

562	4.026952743530273	0.1288
563	3.7012569904327393	0.1161
564	3.9194936752319336	0.1258
565	3.9520022869110107	0.1287
566	3.6345441341400146	0.1138
567	3.899383068084717	0.1072
568	4.044710636138916	0.1083
569	3.638409376144409	0.1269
570	3.497030019760132	0.14
571	3.4805984497070312	0.1396
572	3.638235569000244	0.1509
573	3.4710140228271484	0.113
574	3.617884874343872	0.1044
575	3.409316062927246	0.1212
576	4.45103645324707	0.1242
577	3.4970669746398926	0.1239
578	3.3639256954193115	0.1421
579	4.9568657875061035	0.1146
580	3.9203896522521973	0.1337
581	3.8524386882781982	0.146
582	3.6283345222473145	0.1085
583	3.6385562419891357	0.096
584	3.705982208251953	0.1192
585	3.5681405067443848	0.1334
586	3.7330286502838135	0.1329
587	3.6243698596954346	0.115
588	3.38537859916687	0.1223
589	3.7507612705230713	0.1216
590	3.3929483890533447	0.1184
591	3.860651969909668	0.1341
592	2.9980154037475586	0.1315
593	4.1174726486206055	0.1327
594	3.728353977203369	0.1265
595	4.090984344482422	0.1372
59

842	3.574740171432495	0.1106
843	3.537802219390869	0.1079
844	7.040840148925781	0.0944
845	4.069217681884766	0.104
846	3.348811149597168	0.0935
847	3.695591926574707	0.1206
848	3.8270301818847656	0.1011
849	3.8808112144470215	0.1365
850	3.49353289604187	0.1263
851	3.6216282844543457	0.1278
852	3.261204242706299	0.1332
853	3.35603666305542	0.1339
854	3.7005863189697266	0.1251
855	3.9174134731292725	0.1088
856	3.4096601009368896	0.1229
857	3.904186248779297	0.1366
858	3.3514020442962646	0.1401
859	4.511240005493164	0.1239
860	4.234770774841309	0.1205
861	4.186304569244385	0.1551
862	4.093225955963135	0.1288
863	3.710198402404785	0.1189
864	3.616549015045166	0.1462
865	3.551891803741455	0.1349
866	3.7881581783294678	0.1336
867	3.323939561843872	0.1244
868	3.6540143489837646	0.1142
869	3.884939193725586	0.1357
870	3.5279858112335205	0.1024
871	3.713823080062866	0.1145
872	3.917747974395752	0.147
873	4.324295520782471	0.1444
874	4.28816556930542	0.1314
875	3.5879554748535156	0.1184
876	3.79

ValueError: could not convert string to float: '(most'

In [ ]:
torch.save(model.state_dict(), f'{load_vocab}/model_weights')

# x = torch.randn(4,6,1)
# print (x)
# x = x.view(4, -1)
# print (x)
# # print(x)
# # probs, ix = x[:, :].data.topk(1)
# # print(probs)
# # print(ix)

# x = torch.randn(3,2)
# print(x)
# y = torch.randn(3,2)
# print(y)
# print (((x - y)**2).mean())
# #F.cosine_embedding_loss(x, y, reduction='mean')

In [ ]:
print(SRC.vocab.itos[4880])